In [95]:
import torch
import torch.nn as nn
import torch.optim as optim

In [96]:
class DRLModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DRLModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [97]:
# Define a function to convert input data to tensors
def preprocess_input(input_data, topology_data):
    input_tensors = {}
    for topology, data in input_data.items():
        flow_demands = data['flow_demands']
        
        # Convert flow demands to tensor
        flow_demands_tensor = torch.tensor([[ord(src) - ord('A'), ord(dest) - ord('A'), demand] for src, dest, demand in flow_demands], dtype=torch.float32)
        
        # Get topology features
        topology_index = int(topology.split('_')[-1]) - 1
        topology_features = topology_data[topology_index]
        
        # Concatenate all features
        input_tensors[topology] = (flow_demands_tensor.view(-1), topology_features)
    
    return input_tensors

In [98]:
def preprocess_output(output_data):
    output_tensors = {}
    for topology, data in output_data.items():
        success_counts = [data[i]['success_count'] for i in range(1, 6)]  # Extract success counts
        output_tensors[topology] = torch.tensor(success_counts, dtype=torch.float32)
    return output_tensors

In [99]:
def train_drl_model(model, optimizer, num_epochs, input_data, output_data):
    for epoch in range(num_epochs):
        total_loss = 0.0
        for topology, (features, topology_features) in input_data.items():
            optimizer.zero_grad()
            predicted_output = model(torch.cat((features, topology_features)))
            loss = nn.MSELoss()(predicted_output, output_data[topology])
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
        if epoch % 100 == 0:
            print(f'Epoch [{epoch}/{num_epochs}], Total Loss: {total_loss}')

In [100]:
# Read topology files and extract topology features
def read_topology_file(filepath):
    with open(filepath, 'r') as file:
        content = file.readlines()
        nodes = [line.strip() for line in content[1:content.index('Edges:\n')]]
        edges = [line.split() for line in content[content.index('Edges:\n')+1:]]
    return nodes, edges

In [101]:
def extract_topology_features(nodes, edges):
    return torch.tensor([len(nodes), len(edges)], dtype=torch.float32)


In [102]:
# Load topology data
filepaths = [
    "M:/Group Project/ManojTopologies/topology_1.txt",
    "M:/Group Project/ManojTopologies/topology_2.txt",
    "M:/Group Project/ManojTopologies/topology_3.txt",
    "M:/Group Project/ManojTopologies/topology_4.txt",
    "M:/Group Project/ManojTopologies/topology_5.txt",    
]

topology_data = []
for filepath in filepaths:
    nodes, edges = read_topology_file(filepath)
    topology_data.append(extract_topology_features(nodes, edges))

# Define input and output data
input_data = {
    'Topology_1': {'flow_demands': [('F', 'E', 17), ('F', 'A', 10), ('B', 'A', 46), ('A', 'C', 29), ('B', 'E', 50)]},
    'Topology_2': {'flow_demands': [('F', 'J', 16), ('J', 'E', 17), ('E', 'A', 31), ('D', 'E', 19), ('E', 'A', 43)]},
    'Topology_3': {'flow_demands': [('A', 'I', 21), ('D', 'E', 14), ('D', 'G', 45), ('H', 'F', 33), ('H', 'B', 47)]},
    'Topology_4': {'flow_demands': [('B', 'C', 41), ('A', 'F', 43), ('C', 'B', 27), ('E', 'B', 41), ('C', 'E', 40)]},
    'Topology_5': {'flow_demands': [('D', 'C', 45), ('C', 'I', 33), ('C', 'A', 31), ('J', 'D', 46), ('J', 'K', 50)]}
}

output_data = {
    'Topology_1': {1: {'success_count': 4}, 2: {'success_count': 3}, 3: {'success_count': 3}, 4: {'success_count': 3}, 5: {'success_count': 3}},
    'Topology_2': {1: {'success_count': 2}, 2: {'success_count': 2}, 3: {'success_count': 1}, 4: {'success_count': 1}, 5: {'success_count': 1}},
    'Topology_3': {1: {'success_count': 4}, 2: {'success_count': 3}, 3: {'success_count': 4}, 4: {'success_count': 4}, 5: {'success_count': 3}},
    'Topology_4': {1: {'success_count': 0}, 2: {'success_count': 0}, 3: {'success_count': 0}, 4: {'success_count': 0}, 5: {'success_count': 0}},
    'Topology_5': {1: {'success_count': 5}, 2: {'success_count': 5}, 3: {'success_count': 5}, 4: {'success_count': 5}, 5: {'success_count': 5}}
}

input_dim = len(next(iter(input_data.values()))['flow_demands']) * 3 + 2  # Adding 2 for topology features
output_dim = 5  # Number of successful demand flows
model = DRLModel(input_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 1000

# Preprocess input and output data
input_tensors = preprocess_input(input_data, topology_data)
output_tensors = preprocess_output(output_data)

# Train the DRL model
train_drl_model(model, optimizer, num_epochs, input_tensors, output_tensors)

Epoch [0/1000], Total Loss: 41.2669837474823
Epoch [100/1000], Total Loss: 0.012689722585491836
Epoch [200/1000], Total Loss: 3.277276874769086e-06
Epoch [300/1000], Total Loss: 0.0022644832206424326
Epoch [400/1000], Total Loss: 9.920696837813214e-11
Epoch [500/1000], Total Loss: 4.257955208038233e-13
Epoch [600/1000], Total Loss: 2.2959966998180336e-13
Epoch [700/1000], Total Loss: 2.457349761897376e-06
Epoch [800/1000], Total Loss: 1.6259640050922224e-12
Epoch [900/1000], Total Loss: 7.61418730003987e-14


In [103]:
# Function to preprocess input data for testing
def preprocess_test_input(test_input_data, test_topology_data):
    test_input_tensors = {}
    for topology, data in test_input_data.items():
        flow_demands = data['flow_demands']
        
        # Convert flow demands to tensor
        flow_demands_tensor = torch.tensor([[ord(src) - ord('A'), ord(dest) - ord('A'), demand] for src, dest, demand in flow_demands], dtype=torch.float32)
        
        # Get topology features
        topology_index = int(topology.split('_')[-1]) - 1
        topology_features = test_topology_data[topology_index]
        
        # Concatenate all features
        test_input_tensors[topology] = (flow_demands_tensor.view(-1), topology_features)
    
    return test_input_tensors

In [104]:
# Function to test the DRL model and calculate optimal number of middle points
def test_drl_model(model, test_input_data, test_topology_data):
    predictions = {}
    for topology, (features, topology_features) in test_input_data.items():
        with torch.no_grad():
            predicted_output = model(torch.cat((features, topology_features)))
        predictions[topology] = predicted_output.argmax().item() + 1  # Select the action with 
        #predictions[topology] = predicted_output.tolist()
    return predictions


In [105]:
def load_filepaths(filepaths):
    topology_data = []
    for filepath in filepaths:
        nodes, edges = read_topology_file(filepath)
        topology_data.append(extract_topology_features(nodes, edges))
    return topology_data

In [106]:
# Define filepaths of the new topologies
filepathtest = [
    "M:/Group Project/Topology_new/Topology_new/topology_1.txt",
    "M:/Group Project/Topology_new/Topology_new/topology_2.txt",
    "M:/Group Project/Topology_new/Topology_new/topology_3.txt",
    "M:/Group Project/Topology_new/Topology_new/topology_4.txt",
    "M:/Group Project/Topology_new/Topology_new/topology_5.txt",  
]

# Test input data
input_data_test = {
    'Topology_1': {'flow_demands': [('K', 'D', 30), ('I', 'O', 31), ('K', 'O', 41), ('K', 'D', 45), ('R', 'G', 40)]},
    'Topology_2': {'flow_demands': [('G', 'O', 12), ('J', 'I', 15), ('R', 'B', 45), ('P', 'E', 30), ('R', 'C', 10)]},
    'Topology_3': {'flow_demands': [('C', 'F', 29), ('I', 'C', 47), ('B', 'I', 11), ('G', 'B', 16), ('I', 'A', 31)]},
    'Topology_4': {'flow_demands': [('A', 'B', 19), ('K', 'L', 43), ('K', 'F', 39), ('L', 'J', 50), ('G', 'E', 35)]},
    'Topology_5': {'flow_demands': [('K', 'A', 22), ('J', 'L', 24), ('E', 'B', 10), ('B', 'E', 12), ('H', 'L', 15)]}
}

# Load topology data from filepathtest
topology_data_test = load_filepaths(filepathtest)

# Preprocess test input data
test_input_tensors = preprocess_test_input(input_data_test, topology_data)

# Test the DRL model
predictions = test_drl_model(model, test_input_tensors, topology_data)

# Print predictions
for topology, prediction in predictions.items():
    print(f"Predictions for {topology}: {prediction}")

Predictions for Topology_1: 1
Predictions for Topology_2: 1
Predictions for Topology_3: 2
Predictions for Topology_4: 5
Predictions for Topology_5: 5


In [107]:
# Function to calculate prediction accuracy
def calculate_accuracy(predictions, actual_output_data):
    correct_predictions = 0
    total_predictions = 0
    for topology, prediction in predictions.items():
        actual_success_counts = [actual_output_data[topology][i]['success_count'] for i in range(1, 6)]
        actual_max_index = actual_success_counts.index(max(actual_success_counts)) + 1
        if prediction == actual_max_index:
            correct_predictions += 1
        total_predictions += 1
    accuracy = correct_predictions / total_predictions
    return accuracy

In [108]:
# Calculate accuracy
accuracy = calculate_accuracy(predictions, output_data)
print("Prediction Accuracy:", accuracy)

Prediction Accuracy: 0.4
